In [16]:
import os
import tqdm
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
from tfcf.metrics import mae
from tfcf.metrics import rmse
from tfcf.datasets import ml1m
from tfcf.config import Config
from tfcf.models.svd import SVD
# from tfcf.models.svd import SVDPP
# from sklearn.model_selection import train_test_split

dir_ = '../../data/'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

In [6]:
# Note that x is a 2D numpy array, 
# x[i, :] contains the user-item pair, and y[i] is the corresponding rating.

df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))

x_train = np.loadtxt(os.path.join(dir_, 'train_x_' + file_name), delimiter=',')
y_train = np.loadtxt(os.path.join(dir_, 'train_y_' + file_name), delimiter=',')

In [8]:
x_test = np.loadtxt(os.path.join(dir_, 'x_test_' + file_name), delimiter=',')

In [9]:
x_train

array([[  746., 36126.],
       [  174., 92702.],
       [  369., 63154.],
       ...,
       [  898., 17446.],
       [  610., 15769.],
       [  585., 27452.]])

In [10]:
x_test

array([[4.31000e+02, 5.17470e+04],
       [5.18000e+02, 1.11400e+05],
       [8.71000e+02, 2.50150e+04],
       ...,
       [7.64000e+02, 3.39560e+04],
       [8.10000e+01, 1.47770e+04],
       [1.52000e+02, 1.26473e+05]])

In [11]:
config = Config()
config.num_users = len(df['uid'].unique())
config.num_items = len(df['tid'].unique())
config.min_value = df['rating'].min()
config.max_value = df['rating'].max()

In [ ]:
# For top N
pd_train = pd.read_pickle(os.path.join(dir_, 'train_' + file_name[:-3] + 'pkl'))
pd_all = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))

x_test = []
num_users = len(pd_all['uid'].unique())
num_tracks = len(pd_all['tid'].unique())
print(num_users,num_tracks)

for i in tqdm(range(num_users)):
    
    user = pd_train[pd_train['uid']==i]
    
    for j in range(num_tracks):
        r = user[user['tid']==j]['rating']
        if len(r) == 0:
            x_test.append([i, j])

953 157565


In [6]:
np.savetxt(os.path.join(dir_, 'x_test_' + file_name), x_test, delimiter=",")
x_test

array([[0.000e+00, 0.000e+00],
       [0.000e+00, 1.000e+00],
       [0.000e+00, 3.000e+00],
       ...,
       [2.100e+02, 8.937e+03],
       [2.100e+02, 8.939e+03],
       [2.100e+02, 8.940e+03]])

In [7]:
with tf.Session() as sess:
    # For SVD++ algorithm, if `dual` is True, then the dual term of items' 
    # implicit feedback will be added into the original SVD++ algorithm.
    # model = SVDPP(config, sess, dual=False)
    # model = SVDPP(config, sess, dual=True)
    model = SVD(config, sess)
#     model.train(x_train, y_train, validation_data=(
#         x_test, y_test), epochs=5, batch_size=1024)

    model.train(x_train, y_train, epochs=5, batch_size=1024)
        
    y_pred = model.predict(x_test)
#     print('rmse: {}, mae: {}'.format(rmse(y_test, y_pred), mae(y_test, y_pred)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
145/145 [==============================] - 1s 9ms/step - rmse: 1.6982 - mae: 1.5206
Epoch 2/5
145/145 [==============================] - 1s 7ms/step - rmse: 1.6808 - mae: 1.5056
Epoch 3/5
145/145 [==============================] - 1s 8ms/step - rmse: 1.6577 - mae: 1.4850
Epoch 4/5
145/145 [==============================] - 1s 9ms/step - rmse: 1.6219 - mae: 1.4518
Epoch 5/5
145/145 [==============================] - 1s 7ms/step - rmse: 1.5747 - mae: 1.4071


In [8]:
print(y_pred)

[2.274596  2.3388152 2.3375711 ... 2.3952677 2.0239165 2.207427 ]


In [9]:
df = pd.DataFrame(y_pred, columns=['rating'])
df2 = pd.DataFrame(x_test, columns=['uid', 'tid'])
df2.insert(2, 'rating', y_pred, False) 
df2

,uid,tid,rating
0,0.0,0.0,2.274596
1,0.0,1.0,2.338815
2,0.0,3.0,2.337571
3,0.0,4.0,2.160373
4,0.0,6.0,2.226491
...,...,...,...
1738858,210.0,8935.0,2.193288
1738859,210.0,8936.0,2.302076
1738860,210.0,8937.0,2.395268
1738861,210.0,8939.0,2.023916


In [10]:
df2.to_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))